In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from datetime import datetime
import time

# Задание 1

### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
# Создаём функцию, которая собирает с сайта habr.com/ru ссылки на все посты материалы (статьи, посты),
# в которых содержится указанное ключевое слово

def get_all_links(query, pages):
    
    links = []
    
    for page in range(1, pages+1):
        
        URL = 'https://habr.com/ru/search/' + 'page' + str(page)
        params = {
        'q': str(query)
        }

        res = requests.get(URL, params)
        soup = BeautifulSoup(res.text, 'html.parser')
        articles = soup.find_all('article')
        list_of_links = list(map(lambda x: x.find('a', class_='post__title_link').get('href'), articles))
        links.extend(list_of_links)
        
    return links

In [ ]:
%%time

# Применяем данную функцию для ключевых слов из списка keywords

keywords = ['python', 'парсинг']

links = []

for keyword in keywords:
    links.extend(get_all_links(keyword, 50))
    
links

In [ ]:
%%time

# и теперь в цикле для каждой ссылки из списка извлекаем и добавляем в датафрейм искомую информацию:

df = pd.DataFrame()

for link in links:
    
    request = requests.get(link)
    link_soup = BeautifulSoup(request.text, 'html.parser')
    
    # опытным путём обнаружено, что дата и заголовок хранятся в словаре, который можно спасрсить следующим образом:    
    
    article_dict = json.loads(''.join(link_soup.find('script', {'type': 'application/ld+json'}).contents))
    
    date = datetime.strptime(article_dict['datePublished'], '%Y-%m-%dT%H:%M:%S%z')
    headline = article_dict['headline']
    
    # опытным путём обнаружено, что текст статьи на страницах размечен по-разному, поэтому его парсим по-разному:
    
    text = []
    
    if link_soup.find('div', class_ = 'post__text post__text_v1'):
        text = link_soup.find('div', class_ = 'post__text post__text_v1').text
    
    if link_soup.find('div', class_ = 'post__text post__text_v2'):
        text = link_soup.find('div', class_ = 'post__text post__text_v2').text
        
    if link_soup.find('div', class_ = 'post__text post__text-html post__text_v1'):
        text = link_soup.find('div', class_ = 'post__text post__text-html post__text_v1').text
        
    row = {'post_date': date, 
           'headline': headline, 
           'link': link, 
           'text': text }
    
    df = pd.concat([df, pd.DataFrame([row])])
    
df

# Задание 2

Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [ ]:
emails = ['nmmmm@xaker.ru', 'sa@mail.ru']

In [ ]:
df_leaks = pd.DataFrame()

for email in emails:
    
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

    params = { 'emailAddresses': [email] }
    
    headers = {'Vaar-Version': '0',
               'Vaar-Header-App-Product': 'hackcheck-web-avast',
               'Content-Type': 'application/json;charset=UTF-8',
               'Host': 'identityprotection.avast.com',
               'Origin': 'https://www.avast.com',
               'Referer': 'https://www.avast.com/',
               'vaar-header-captcha-response-token': '03AGdBq26r4LzketNJyAZwqhNn_xUvpCDVBWeqSi22LGFuHGtYXmmvuMaM0SyDB5z4ZMkSpXguHvoTssuWm_FrQeB9gaNmGcIupugGIoz1c2eJbMJXHfXnG-BS6XnldZBVWX4MkyGsDkc0wPsgjbqy8VwlBphCqP87nEbtYiXUTa6NQbpMJYlgGyHMf5sXNMvPAiXhm1SnDkQbSY9uBZ_34VSxUYgKNJJ6C65kc-qnnxWa7qJr5rcy0ofdR9tNLWQWqxiZG49joN0o3QYHjqR76XrM-SpyYue2UhILbYAKdVGg5SrwBhYWd5sPXepeV64eChO8vQc6WeDyriLBUAHvhpCuGD1yCabCtRBtYZnMur0NqGZ9pLYFlWJR8x_DwUfYPv6ARLpqM74gwjYj75mdTVjv195wSDhL9xUWKv7CsBmqaKg_MezbtXTVkSPjoSz0cu_z1L6GA4Vg' }

    req = requests.post(URL, json=params, headers=headers)
    response_soup = BeautifulSoup(req.text, 'html.parser')
    response_json = json.loads(str(response_soup))
    
    for breachID in response_json['summary'][str(email)]['breaches']:
        
        e_mail = email
        date = datetime.strptime(response_json['breaches'][str(breachID)]['publishDate'], '%Y-%m-%dT%H:%M:%S%z')
        website = response_json['breaches'][str(breachID)]['site']
        comment = response_json['breaches'][str(breachID)]['description']

        row = {'email': e_mail,
               'date': date,
               'website': website,
               'comment': comment }

        df_leaks = pd.concat([df_leaks, pd.DataFrame([row])])

df_leaks 